In [1]:
import os
import re
import google.generativeai as genai

In [ ]:
# Gemini API key 都放進來
key_list = [
            
            ]

In [ ]:
# 定義來源與輸出資料夾
SOURCE_DIR = ""
OUTPUT_DIR = ""
OUTPUT_DIR_GEN = ""
os.makedirs(OUTPUT_DIR, exist_ok=True)
os.makedirs(OUTPUT_DIR_GEN, exist_ok=True)

# 目前目錄生產到的檔案編號(從0開始)
# AI generate 會用到而已
file_num = 0

In [ ]:
# 過濾掉劣質data以及刪除粗體的*符號

def remove_asterisks(input_string):
    return input_string.replace('*', '')

def check_string(s):
    letter_count = sum(('A' <= c <= 'Z') or ('a' <= c <= 'z') for c in s)
    digit_count = sum(c.isdigit() for c in s)

    if letter_count > 500 or digit_count > 300:
        return False
    else:
        return True

In [ ]:
# AI paraphrase
# 根據論文原文抓出來的片段進行改寫

# 正則表達式匹配 "數字.txt" 格式的檔案
pattern = re.compile(r"^\d+\.txt$")
GENAI_API_KEY = key_list[0]
key_index = 0

# 遍歷來源資料夾的所有檔案
for filename in os.listdir(SOURCE_DIR):
    if pattern.match(filename):
        file_path = os.path.join(SOURCE_DIR, filename)
        with open(file_path, "r", encoding="utf-8") as f:
            content = f.read()

        try:
            # 呼叫 Gemini API 進行改寫
            genai.configure(api_key=GENAI_API_KEY)
            model = genai.GenerativeModel("gemini-1.5-pro")
            response = model.generate_content("請改寫以下內容，不要回傳改寫以外的內容:" + content)
            rewritten_text = response.text
        except:
            key_index = key_index + 1
            if key_index >= len(key_list):
                break
            GENAI_API_KEY = key_list[key_index]

            # 呼叫 Gemini API 進行改寫
            genai.configure(api_key=GENAI_API_KEY)
            model = genai.GenerativeModel("gemini-1.5-pro")
            response = model.generate_content("請改寫以下內容，不要回傳改寫以外的內容:" + content)
            rewritten_text = response.text

        # 儲存到 paraphrase 資料夾
        output_path = os.path.join(OUTPUT_DIR, filename)
        with open(output_path, "w", encoding="utf-8") as f:
            f.write(rewritten_text)

        os.remove(file_path)

        print(f"已處理: {filename} -> {output_path}")

In [ ]:
# 根據 extract.py 產生的 index.txt ，將同一篇論文的前後文編號存到一組 list 裡面

INDEX_FILE = os.path.join(SOURCE_DIR, "index.txt")

def parse_index_file(filepath):
    result = []

    if not os.path.exists(filepath):
        print(f"{filepath} 不存在。")
        return result

    with open(filepath, 'r', encoding='utf-8') as f:
        for line in f:
            line = line.strip()
            if not line:
                continue
            try:
                parts = line.split(':')
                if len(parts) != 2:
                    continue

                txt_range = parts[1].strip()
                start_txt, end_txt = txt_range.split('-')
                start_num = int(start_txt.strip().replace('.txt', ''))
                end_num = int(end_txt.strip().replace('.txt', ''))

                if start_num < end_num:
                    result.append([start_num, end_num])
            except Exception as e:
                print(f"解析失敗: {line}，錯誤: {e}")
    
    return result

data = parse_index_file(INDEX_FILE)
print("結果陣列：")
print(data)

In [ ]:
# AI generation
# 根據免篇論文的前後文，讓Gemini生成中間段落

GENAI_API_KEY = key_list[0]
key_index = 0

# 遍歷來源資料夾的所有檔案
for pair in data:
    file_path_start = os.path.join(SOURCE_DIR, str(pair[0]) + ".txt")
    file_path_end = os.path.join(SOURCE_DIR, str(pair[1]) + ".txt")
    
    with open(file_path_start, "r", encoding="utf-8") as f:
        content_start = f.read()

    with open(file_path_end, "r", encoding="utf-8") as f:
        content_end = f.read()
        
    try:
        # 呼叫 Gemini API 進行生成
        genai.configure(api_key=GENAI_API_KEY)
        model = genai.GenerativeModel("gemini-1.5-pro")
        response = model.generate_content("以下兩段文章為同一篇論文中前面章節與後面章節內容抓出來的小片段，請生成一個2000字以上類似的小片段，內容為你認為這兩段文章中間應該要有的文字，不要回傳標題以及與內容無關的文字，也不要使用粗體排版或標標題。\n文章一:\n" + content_start + "\n文章二:\n" + content_end)
        rewritten_text = response.text
        if rewritten_text.count('*') > 4:
            rewritten_text = remove_asterisks(rewritten_text)
    except:
        key_index = key_index + 1
        if key_index >= len(key_list):
            break
        GENAI_API_KEY = key_list[key_index]

        # 呼叫 Gemini API 進行生成
        genai.configure(api_key=GENAI_API_KEY)
        model = genai.GenerativeModel("gemini-1.5-pro")
        response = model.generate_content("以下兩段文章為同一篇論文中前面章節與後面章節內容抓出來的小片段，請生成一個2000字以上類似的小片段，內容為你認為這兩段文章中間應該要有的文字，不要回傳標題以及與內容無關的文字，也不要使用粗體排版或標標題，。" + "\n文章一:\n" + content_start + "\n文章二:\n" + content_end)
        rewritten_text = response.text
        if rewritten_text.count('*') > 4:
            rewritten_text = remove_asterisks(rewritten_text)

    # 儲存到 generate 資料夾
    if check_string(rewritten_text):
        filename_num = filename_num + 1
        filename = str(filename_num) + ".txt"
        output_path = os.path.join(OUTPUT_DIR, filename)
        with open(output_path, "w", encoding="utf-8") as f:
            f.write(rewritten_text)

        print(f"已處理: {filename} -> {output_path}")    

已處理: 263.txt -> D:\py\plag\paper\generate\ncu_2019\263.txt
已處理: 264.txt -> D:\py\plag\paper\generate\ncu_2019\264.txt
已處理: 265.txt -> D:\py\plag\paper\generate\ncu_2019\265.txt
已處理: 266.txt -> D:\py\plag\paper\generate\ncu_2019\266.txt
已處理: 267.txt -> D:\py\plag\paper\generate\ncu_2019\267.txt
已處理: 268.txt -> D:\py\plag\paper\generate\ncu_2019\268.txt
已處理: 269.txt -> D:\py\plag\paper\generate\ncu_2019\269.txt
已處理: 270.txt -> D:\py\plag\paper\generate\ncu_2019\270.txt
已處理: 271.txt -> D:\py\plag\paper\generate\ncu_2019\271.txt
已處理: 272.txt -> D:\py\plag\paper\generate\ncu_2019\272.txt
已處理: 273.txt -> D:\py\plag\paper\generate\ncu_2019\273.txt
已處理: 274.txt -> D:\py\plag\paper\generate\ncu_2019\274.txt
已處理: 275.txt -> D:\py\plag\paper\generate\ncu_2019\275.txt
已處理: 276.txt -> D:\py\plag\paper\generate\ncu_2019\276.txt
已處理: 277.txt -> D:\py\plag\paper\generate\ncu_2019\277.txt
已處理: 278.txt -> D:\py\plag\paper\generate\ncu_2019\278.txt
已處理: 279.txt -> D:\py\plag\paper\generate\ncu_2019\279.t